In [7]:
# display results
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer
renderer = JupyterRenderer()

In [2]:
import random
import time
import sys

from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeCylinder
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Ax2, gp_Dir
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.Core.TopTools import TopTools_ListOfShape

In [4]:
def random_pnt():
    x, y, z = [random.uniform(0, 1) for i in range(3)]
    return gp_Pnt(x, y, z)

def random_vec():
    x, y, z = [random.uniform(-1, 1) for i in range(3)]
    return gp_Vec(x, y, z)

def fuzzy_cut(shape_A, shape_B, tol=5e-5, parallel=False):
    """ returns shape_A - shape_B
    """
    cut = BRepAlgoAPI_Cut()
    L1 = TopTools_ListOfShape()
    L1.Append(shape_A)
    L2 = TopTools_ListOfShape()
    L2.Append(shape_B)
    cut.SetArguments(L1)
    cut.SetTools(L2)
    cut.SetFuzzyValue(tol)
    cut.SetRunParallel(parallel)
    cut.Build()
    return cut.Shape()

def emmenthaler(event=None):
    init_time = time.time()
    scope = 200.
    nb_iter = 40
    box = BRepPrimAPI_MakeBox(scope, scope, scope).Shape()

    def do_cyl():
        axe = gp_Ax2()
        axe.SetLocation(gp_Pnt((random_vec()*scope).XYZ()))
        axe.SetDirection(gp_Dir(random_vec()))
        cyl = BRepPrimAPI_MakeCylinder(axe, random.uniform(8, 36), 5000.)
        return cyl.Shape()

    # perform a recursive fusszy cut
    # initialize the loop with the box shape
    shp = box
    for i in range(nb_iter):
        cyl = do_cyl()
        tA = time.time()
        shp = fuzzy_cut(shp, cyl, 1e-4)
        print('boolean cylinder:', i, 'took', time.time()-tA)
    
    total_time = time.time() - init_time
    print("Total time : %fs" % total_time)
    return shp

In [5]:
shape = emmenthaler()

boolean cylinder: 0 took 0.002116680145263672
boolean cylinder: 1 took 0.0032830238342285156
boolean cylinder: 2 took 0.0013718605041503906
boolean cylinder: 3 took 0.0013539791107177734
boolean cylinder: 4 took 0.004488945007324219
boolean cylinder: 5 took 0.0012171268463134766
boolean cylinder: 6 took 0.0026390552520751953
boolean cylinder: 7 took 0.0012569427490234375
boolean cylinder: 8 took 0.0013060569763183594
boolean cylinder: 9 took 0.013536930084228516
boolean cylinder: 10 took 0.01859569549560547
boolean cylinder: 11 took 0.0020110607147216797
boolean cylinder: 12 took 0.0019960403442382812
boolean cylinder: 13 took 0.002022266387939453
boolean cylinder: 14 took 0.061964988708496094
boolean cylinder: 15 took 0.00833892822265625
boolean cylinder: 16 took 0.0027599334716796875
boolean cylinder: 17 took 0.003770112991333008
boolean cylinder: 18 took 0.002665996551513672
boolean cylinder: 19 took 0.0025970935821533203
boolean cylinder: 20 took 0.0049169063568115234
boolean cylin

In [6]:
shape

<class 'TopoDS_Compound'>

In [8]:
renderer.DisplayShape(shape, render_edges=True, topo_level="default", shape_color="#abdda4", update=True)